### Pre-requisitos para que funcione correctamente
<ul>
    <li>pip install ibm_db</li>
    <li> Instalar nueva version DB2 11.5 del <a href="https://w3-01.ibm.com/download/standardsoftware/PC/lang_en/issiCatalogPC.html" target="_blank">pc@catalog</a></li>
    <li> Usar version modificada del .bat setup_cmd para setear las conexiones (la tengo yo)</li>
    <li> Carpeta utils, tiene que tener licencia 11.5 (la tengo yo) y los GSKit nuevos version 8.50.89 al 27/4</li>
    <li> El conection string, además de lo que figura en github --> <a href="https://github.com/ibmdb/python-ibmdb#api" target="_blank">IBM_DB API</a>, tiene que tener los parametros security=ssl,SSLClientKeystoredb=<b>Dir ibmca.kdb</b>;SSLClientKeystoreDBPassword=<b>Psw .kdb (.bat)</b></li>
    <li>hostname, database name, port datos claves que tienen que estar bien en ODBC</li>
</ul>

##### IBM_DB has proven not work for ICFS connection

In [3]:
import ibm_db
import ibm_db_dbi as dbi
import pandas as pd
import numpy as np
import time
import pickle


start = time.time()

inputfile=r"C:\Users\016434613\Desktop\Todo\input_PyDB2.txt"
datalist = []
SSLClientKeystoredb_path="C:\ProgramData\IBM\DB2\DB2COPY1\DB2\ibmca.kdb"


with open("cred_db","rb") as f:
    cred_dict = pickle.load(f)

uid=cred_dict['user']
psw=cred_dict['psw']
SSLClientKeystoredb=SSLClientKeystoredb_path
SSLClientKeystoreDBPassword=cred_dict['SSLClientKeystoreDBPassword']


''' Usando dbi, podemos usar el metodo de pandas read_sql y tener un dataframe directo del resultado '''

conn=dbi.connect("DATABASE=USIBMVRDP1H;HOSTNAME=usibmvrdp1h.ssiplex.pok.ibm.com;PORT=5520;PROTOCOL=TCPIP;UID="+uid+";PWD="+psw+";Security=SSL;SSLClientKeystoredb="+SSLClientKeystoredb+";SSLClientKeystoreDBPassword="+SSLClientKeystoreDBPassword+";", "", "")



sql = ''' SELECT CTY, DIV, LC, TOLI, MAJ, MINOR, SMIN, LERU, FID, SRC, ACCID, ACCTMO,
             ACCTYEAR, AMTLOC, AMTDLR, AMTLOC_REP, AMTDLR_REP, VOUCHER_NBR,
             OEM_PROD_ID, DESCR1, LOCFLD1
          FROM LRPCC.LADETAIL_PM_D
          WHERE (CTY = '613' AND MAJ = '601' AND LERU IN ('001977')
             AND LC IN ('01', '04')) '''



df2=pd.read_sql(sql,conn)

df_Minor = pd.read_excel(r"C:\Users\016434613\Desktop\Todo\Minor Codes.xlsx",sheet_name="REF (2)",converters={'Minor':str})
df_Minor = df_Minor.loc[:,['Minor','Desc']]
df_Minor = df_Minor.rename(columns={'Minor':'MINOR'})
df2 = df2.merge(df_Minor, on="MINOR", how="left")
df2.to_excel(r"C:\Users\016434613\Desktop\dept1977_exp.xlsx", index=False)



end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Report created in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Report created in: {:.2f} minutes".format(mins)
    print(minsstr)

Report created in: 5.07 seconds


In [4]:
table_ = pd.pivot_table(df2, values=['AMTDLR_REP'], index=['Desc'], columns=['ACCTMO'],
                       aggfunc=np.sum, fill_value=0, margins=True, margins_name="Total")
table_.sort_values(by=('AMTDLR_REP','Total'),ascending=False)

AMTDLR_REP            \
ACCTMO                                                     01        02   
Desc                                                                      
Total                                                52737.18  46393.62   
Salaries                                             26228.57  25791.72   
Social Seurity Taxes                                 10618.83   8243.68   
Vacations Accrual                                     9434.73    487.60   
Bonuses                                               2617.52   2738.05   
Welfare                                                855.10   1705.31   
Variable Pay                                             0.00   4760.46   
Awards                                                1299.80   1195.41   
Other Comp                                             675.77    723.12   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...     171.46   -171.46   
DEFINED CONTRIBUTION EXPENSE                           339.86    323.68   
Overtime                                                 0.00      0.00   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS             227.63    228.67   
Deprec IBM equip-Non Wkst                              163.00    198.00   
Servicios IT - IGA SD                                    0.00      0.00   
NON-WIRELESS REMOTE DSL ACCESS                          38.95     95.62   
Postretirement Plan                                     61.88     68.07   
Retirement Plan                                          4.08      3.88   
Audio Conferencing Products and Services                 0.00      1.81   

                                                                        \
ACCTMO                                                    03        04   
Desc                                                                     
Total                                               44815.14  55955.78   
Salaries                                            23798.32  27783.80   
Social Seurity Taxes                                 8630.75   6013.82   
Vacations Accrual                                    3958.21  12342.50   
Bonuses                                              2729.37   3811.74   
Welfare                                               958.10   1838.78   
Variable Pay                                            0.00      0.00   
Awards                                               1317.13    347.79   
Other Comp                                            682.16    718.88   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...   1498.74    356.29   
DEFINED CONTRIBUTION EXPENSE                          291.49    412.91   
Overtime                                              383.45    566.93   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS            241.51    246.14   
Deprec IBM equip-Non Wkst                             198.00    308.00   
Servicios IT - IGA SD                                   0.00    826.32   
NON-WIRELESS REMOTE DSL ACCESS                         56.34    308.86   
Postretirement Plan                                    68.07     68.07   
Retirement Plan                                         3.50      4.95   
Audio Conferencing Products and Services                0.00      0.00   

                                                               
ACCTMO                                                  Total  
Desc                                                           
Total                                               199901.72  
Salaries                                            103602.41  
Social Seurity Taxes                                 33507.08  
Vacations Accrual                                    26223.04  
Bonuses                                              11896.68  
Welfare                                               5357.29  
Variable Pay                                          4760.46  
Awards                                                4160.13  
Other Comp                                            2799.93  
TRA

#### Para que la conexión de PyOdbc funcione, tuve que cambiar la psw con el config assistant de DB2 y antes tenía una con mayusculas y chars especiales como _ y la cambie por una toda en minusculas sin chars especiales


##### PyODBC me funciono para ICFS mientras que IBM_DB no

In [1]:
import pyodbc
import pandas as pd
import pickle
import time
import os
import numpy as np
from Reports.Create_Credentials import createCredentials

start = time.time()

cr = createCredentials()

#file should be stored in the same path
cred_file = "cred_db"
minor_codes = r"C:\Users\016434613\Desktop\Todo\Minor Codes.xlsx"
outfile = r"C:\Users\016434613\Desktop\dept1977_exp.xlsx"

if os.path.exists(cred_file):
    cred_dict = cr.load(cred_file)

try:
    conn_string="DSN=DP1H;UID=LEDGR97;PWD=juanmanueldel26"
except:
    print("File cred_db does not exists")
conn = pyodbc.connect(conn_string)

sql = ''' SELECT A.CTY, A.FID, A.SEQ, A.DEBIT_AMOUNT, A.CREDIT_AMOUNT, A.RECORD_COUNT, A.TIME_DATE
    FROM CF1DB.CFLVHDR1 A
    WHERE A.CTY||A.FID||A.SEQ NOT IN
    (SELECT B.CTY||B.FID||B.FSEQ FROM CF1DB.CFLVFNR B)
    UNION ALL
    SELECT A.CTY, A.FID, A.SEQ, A.DEBIT_AMOUNT, A.CREDIT_AMOUNT, A.RECORD_COUNT, A.TIME_DATE
    FROM CF2DB.CFLVHDR1 A
    WHERE A.CTY||A.FID||A.SEQ NOT IN
    (SELECT B.CTY||B.FID||B.FSEQ FROM CF2DB.CFLVFNR B)
    ORDER BY FID, CTY, SEQ
    WITH UR '''

df =pd.read_sql(sql,conn)

df

,CTY,FID,SEQ,DEBIT_AMOUNT,CREDIT_AMOUNT,RECORD_COUNT,TIME_DATE
0,655,010,4719,3.774226e+13,3.774226e+13,0000212,20230102095229
1,661,020,4961,2.830653e+12,2.830653e+12,0000333,20230102123940
2,683,020,4946,7.110833e+06,7.110833e+06,0000051,20230102125241
3,815,020,4959,2.975723e+08,2.975723e+08,0000160,20230102125240
4,871,020,4934,3.164049e+06,3.164049e+06,0000003,20230102125242
5,655,035,4206,6.611478e+12,6.611478e+12,0001052,20221231003049
6,661,035,4409,3.133652e+11,3.133652e+11,0000232,20230102095234
7,683,035,4095,7.109480e+05,7.109480e+05,0000012,20230102095239
8,815,035,4298,1.378854e+08,1.378854e+08,0000092,20230102095244
9,683,066,0095,0.000000e+00,0.000000e+00,0001730,20230102110133


In [2]:
table_ = pd.pivot_table(df, values=['AMTDLR_REP'], index=['Desc'], columns=['ACCTMO'],
                       aggfunc=np.sum, fill_value=0, margins=True, margins_name="Total")
table_.sort_values(by=('AMTDLR_REP','Total'),ascending=False)

AMTDLR_REP           \
ACCTMO                                                     01       02   
Desc                                                                     
Total                                                 5273718  4639362   
Salaries                                              2622857  2579172   
Social Seurity Taxes                                  1061883   824368   
Vacations Accrual                                      943473    48760   
Bonuses                                                261752   273805   
Variable Pay                                                0   476046   
Welfare                                                 85510   170531   
Travel                                                      0        0   
Other Comp                                              67577    72312   
Awards                                                 129980   119541   
Servicios IT - IGA SD                                       0        0   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...      17146   -17146   
DEFINED CONTRIBUTION EXPENSE                            33986    32368   
Deprec IBM equip-Non Wkst                               16300    19800   
Other Taxes                                                 0        0   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS              22763    22867   
Overtime                                                    0        0   
Transfer from related WT company                            0        0   
Postretirement Plan                                      6188     6807   
NON-WIRELESS REMOTE DSL ACCESS                           3895     9562   
RECOGNITION EVENTS/CONTESTS                                 0        0   
Transferencia de CC 877                                     0        0   
Cellular Phone Expenses                                     0        0   
Retirement Plan                                           408      388   
Audio Conferencing Products and Services                    0      181   
TRANSFER FROM BUILDING OPERATIONS COST CENTER               0        0   
Office Supplies                                             0        0   

                                                                               \
ACCTMO                                                   03       04       05   
Desc                                                                            
Total                                               4481514  5595578  5347426   
Salaries                                            2379832  2778380  2552342   
Social Seurity Taxes                                 863075   601382  1680532   
Vacations Accrual                                    395821  1234250  -332458   
Bonuses                                              272937   381174   428213   
Variable Pay                                              0        0   508116   
Welfare                                               95810   183878   134752   
Travel                                                    0        0        0   
Other Comp                                            68216    71888    87870   
Awards                                               131713    34779    28163   
Servicios IT - IGA SD                                     0    82632    81841   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...   149874    35629    44518   
DEFINED CONTRIBUTION EXPENSE                          29149    41291    37817   
Deprec IBM equip-Non Wkst                             19800    30800    30300   
Other Taxes                                               0        0        0   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS            24151    24614    25265   
Overtime                                              38345    56693        0   
Transfer from related WT company                          0        0    27208   
Postretirement Plan                                    6807     6807     7458   
NON-WIRELESS REMOTE DSL ACCESS                  

In [3]:
import getpass as gp
from Reports.Create_Credentials import createCredentials

user = input("Enter user id: \n")
psw = gp.getpass("Enter your password: \n")

cr = createCredentials(user, psw)

cr.create('cred_db')

Enter user id: 
 ar16434
Enter your password: 
 ················


Credentials successfully created!
